In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-12-06"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
18460,2024-12-06,Brasil Nbb,11:00,Fortaleza B.C.,Mogi,1.34,2.87,146.5,1.85,1.81,-7.5,2.02,1.67,rsPoPQTH,0.746269,0.348432,0.540541,0.552486,0.094701,0.2,0.4,NaN,NaN,268.152,117.077769,0.436610,261.436,159.190519,0.608908,161.560,394.518,202.24,240.24,0.0,0.0,0.0,0.0,0.513954,0.015456,0.134139,-2.66,-0.532,-0.639098,0.587413,0.4,-0.187413,-5.00,-1.000,-1.870000,0.223284,0.1,-0.123284
18461,2024-12-06,Brasil Nbb,19:30,Unifacisa,São José,1.52,2.29,156.5,1.84,1.82,-4.5,1.94,1.73,ltiJ6IDj,0.657895,0.436681,0.543478,0.549451,0.094576,0.8,0.8,NaN,NaN,182.380,78.101549,0.428235,230.586,196.765390,0.853328,133.472,301.668,115.92,580.50,0.0,1.0,0.0,0.0,0.285812,0.007728,0.080922,-3.80,-0.760,-0.684211,0.705617,0.4,-0.305617,4.06,0.812,1.588670,0.384178,0.3,-0.084178
18462,2024-12-06,Brasil Nbb,20:00,São Paulo,R10 Score Vasco da Gama,1.38,2.72,151.5,1.87,1.79,-6.5,1.88,1.78,db4B26Ht,0.724638,0.367647,0.534759,0.558659,0.092285,0.2,0.6,NaN,NaN,137.224,53.119084,0.387098,154.870,44.837889,0.289520,159.232,150.412,220.22,194.56,0.0,0.0,0.0,0.0,0.462206,0.030912,0.038640,3.75,0.750,0.506667,0.576863,0.5,-0.076863,3.42,0.684,2.514620,0.510116,0.7,0.189884
18463,2024-12-06,Eua Nba,00:00,Golden State Warriors,Houston Rockets,2.45,1.59,218.5,1.84,2.00,3.5,2.01,1.86,UN4IPsmJ,0.408163,0.628931,0.543478,0.500000,0.037094,0.4,0.8,NaN,NaN,220.378,76.327049,0.346346,204.696,61.389693,0.299907,184.322,257.744,247.45,228.66,0.0,0.0,0.0,0.0,0.301045,0.058926,0.054814,-0.28,-0.056,-25.892857,0.654524,0.6,-0.054524,0.24,0.048,12.291667,0.512196,0.6,0.087804
18464,2024-12-06,Eua Nba,21:00,Philadelphia 76ers,Orlando Magic,2.35,1.64,209.5,1.93,1.93,2.5,1.96,1.84,f1d4mouQ,0.425532,0.609756,0.518135,0.518135,0.035288,0.6,0.6,NaN,NaN,222.232,71.297004,0.320822,181.852,60.716952,0.333881,275.432,181.852,240.72,172.78,0.0,0.0,0.0,0.0,0.251652,0.000000,0.044659,-3.70,-0.740,-1.824324,0.494323,0.3,-0.194323,0.89,0.178,3.595506,0.489184,0.5,0.010816
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18565,2024-12-06,Finlândia I Divisão A,10:00,Kipina,Oulu,1.06,7.80,182.5,1.83,1.83,-32.5,1.95,1.72,O2qXPqx3,0.943396,0.128205,0.546448,0.546448,0.071601,0.6,0.8,NaN,NaN,142.300,46.035272,0.323509,237.538,145.531053,0.612664,99.030,313.442,99.99,243.66,0.0,0.0,0.0,0.0,1.075824,0.000000,0.088629,0.43,0.086,0.697674,0.000000,0.0,0.000000,-3.05,-0.610,-11.147541,0.000000,0.0,0.000000
18566,2024-12-06,Finlândia Korisliiga Feminina,10:00,Kouvottaret F,Pyrinto F,1.21,3.74,159.5,1.83,1.83,-10.5,1.95,1.72,0x4N1yGM,0.826446,0.267380,0.546448,0.546448,0.093826,0.4,0.8,NaN,NaN,213.486,250.481973,1.173295,208.504,217.559162,1.043429,381.996,146.686,660.00,122.10,1.0,0.0,0.0,0.0,0.722820,0.000000,0.088629,7.11,1.422,0.147679,0.000000,0.0,0.000000,-0.17,-0.034,-80.588235,0.000000,0.0,0.000000
18567,2024-12-06,Finlândia Divisão I Feminina,10:00,Torpan Pojat Helsinki II F,Puhuttaret F,1.06,6.56,144.5,1.83,1.83,-15.5,1.95,1.72,v7qpEcjD,0.943396,0.152439,0.546448,0.546448,0.095835,0.8,1.0,NaN,NaN,428.196,712.182564,1.663216,134.188,27.062603,0.201677,100.640,164.278,130.80,154.05,0.0,0.0,0.0,0.0,1.020758,0.000000,0.088629,-1.76,-0.352,-0.170455,0.000000,0.0,0.000000,3.29,0.658,8.449848,0.000000,0.0,0.000000
18568,2024-12-06,Reino Unido Troféu Da Slb,16:30,Newca

## Apostas do dia

### Over v0 (LDA)

In [9]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,00:00,Eua Nba,Golden State Warriors,Houston Rockets,218.5,1.84,0.9718,Over
1,22:00,Eua Nba,Chicago Bulls,Indiana Pacers,245.5,1.81,0.9669,Over
2,22:00,Eua Nba,San Antonio Spurs,Sacramento Kings,226.5,1.84,0.8799,Over
